# lấy link các trận đấu qua các mùa giải

In [1]:
# Chay thư viện
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Cấu hình Selenium
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36")

# Khởi tạo trình duyệt Chrome với cấu hình
driver = webdriver.Chrome(options=chrome_options)

# Truy cập trang web
url = "https://www.soccerstats247.com/competitions/england/premier-league/2023-2024/"
driver.get(url)
time.sleep(2)  # Chờ trang web tải

# Danh sách để lưu các liên kết
all_links = []

while True:
    try:
        # Tìm các liên kết theo XPath
        links = driver.find_elements(By.XPATH, '//*[@id="soccerContentPlaceHolder_ucChampionship_ucIndividualGroupStage_ucMatchesFromLeagues_gvMatches"]/tbody/tr/td[4]/a')
        
        # Danh sách tạm để lưu các liên kết trong vòng này
        temp_links = []
        
        for link in links:
            href = link.get_attribute("href")
            if href not in all_links:  # Kiểm tra xem liên kết đã có trong danh sách chưa
                temp_links.append(href)  # Thêm vào danh sách tạm

        # Đảo ngược danh sách tạm và thêm vào all_links
        all_links.extend(reversed(temp_links))

        # Kiểm tra ngày trong URL để xác định nút nào để nhấn
        if "2023-2024" in url:
            # Nhấn vào nút "Next Week" cho 2023-2024
            try:
                next_week_button = driver.find_element(By.XPATH, '//*[@id="soccerContentPlaceHolder_ucChampionship_ucIndividualGroupStage_ucMatchesFromLeagues_hlNextWeek"]')
                driver.execute_script("arguments[0].scrollIntoView(true);", next_week_button)
                time.sleep(1)  # Chờ một chút để cuộn hoàn tất
                
                # Nhấn vào nút "Next Week"
                driver.execute_script("arguments[0].click();", next_week_button)
                time.sleep(2)  # Chờ trang tải hoàn toàn
            except NoSuchElementException:
                print("Nút 'Next Week' không còn tồn tại. Kết thúc quá trình.")
                break
        else:
            # Nhấn vào nút "Previous Week" cho các mùa khác
            try:
                next_week_button = driver.find_element(By.XPATH, '//*[@id="soccerContentPlaceHolder_ucChampionship_ucIndividualGroupStage_ucMatchesFromLeagues_hlPreviousWeek"]')
                driver.execute_script("arguments[0].scrollIntoView(true);", next_week_button)
                time.sleep(1)  # Chờ một chút để cuộn hoàn tất
                
                # Nhấn vào nút "Previous Week"
                driver.execute_script("arguments[0].click();", next_week_button)
                time.sleep(2)  # Chờ trang tải hoàn toàn
            except NoSuchElementException:
                print("Nút 'Previous Week' không còn tồn tại. Kết thúc quá trình.")
                break

    except Exception as e:
        print(f"Kết thúc quá trình, lỗi: {e}")
        break

# ghi ra file theo thứ tự ngược lại
with open("data_crawl/ss21_22_test.txt", "w") as file:
    for link in all_links[::-1]:
        file.write(link + "\n")

# Đóng trình duyệt
driver.quit()


Nút 'Next Week' không còn tồn tại. Kết thúc quá trình.


In [66]:
# đọc hết tất cả các file txt ss19_20, ss20_21, ss21_22, ss22_23, ss23_24 không có kí tự \n
# ghi vào file ss19_24.txt
all_links = []
for file_name in ["ss19_20.txt", "ss20_21.txt", "ss21_22.txt", "ss22_23.txt", "ss23_24.txt"]:
    with open(file_name, "r") as file:
        for line in file:
            all_links.append(line.strip())

with open("data_crawl/ss19_24.txt", "w") as file:
    for link in all_links:
        file.write(link + "\n")        

# lấy thông tin trận đấu

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

In [2]:
def scrape_link(link):
    # Tạo danh sách tạm để lưu dữ liệu cho từng liên kết
    data = {
        "home_name": None,
        "away_name": None,
        "home_score": None,
        "away_score": None,
        "shots_home": None,
        "shots_away": None,
        "shots_on_goal_home": None,
        "shots_on_goal_away": None,
        "passes_home": None,
        "passes_away": None,
        "accurate_passes_home": None,
        "accurate_passes_away": None,
        "fouls_home": None,
        "fouls_away": None,
        "yellow_cards_home": None,
        "yellow_cards_away": None,
        "possession_time_home": None,
        "possession_time_away": None,
        "red_cards_home": None,
        "red_cards_away": None,
        "date": None
    }
    
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(link, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            data["away_name"] = soup.select_one("#soccerContentPlaceHolder_hlTeam2Name").text.strip() if soup.select_one("#soccerContentPlaceHolder_hlTeam2Name") else None
            data["home_name"] = soup.select_one("#soccerContentPlaceHolder_hlTeam1Name").text.strip() if soup.select_one("#soccerContentPlaceHolder_hlTeam1Name") else None
            score_span = soup.select_one("#soccerContentPlaceHolder_lblScore")
            date_elements = soup.select("td[colspan='3']")

            match_stats_table = soup.find('table', class_='gridAlternate matchStatistics hlCenter')
            if match_stats_table:
                rows = match_stats_table.find_all('tr')

                # Lấy thông tin cú sút
                if len(rows) > 1:
                    shots_home = rows[1].find_all('td')[1].text.strip()
                    shots_away = rows[1].find_all('td')[2].text.strip()
                    data["shots_home"] = int(shots_home) if shots_home.isdigit() else None
                    data["shots_away"] = int(shots_away) if shots_away.isdigit() else None

                # Lấy thông tin cú sút trúng đích
                if len(rows) > 2:
                    shots_on_goal_home = rows[2].find_all('td')[1].text.strip()
                    shots_on_goal_away = rows[2].find_all('td')[2].text.strip()
                    data["shots_on_goal_home"] = int(shots_on_goal_home) if shots_on_goal_home.isdigit() else None
                    data["shots_on_goal_away"] = int(shots_on_goal_away) if shots_on_goal_away.isdigit() else None

                # Lấy thông tin đường chuyền
                if len(rows) > 7:
                    passes_home = rows[7].find_all('td')[1].text.strip()
                    passes_away = rows[7].find_all('td')[2].text.strip()
                    data["passes_home"] = int(passes_home) if passes_home.isdigit() else None
                    data["passes_away"] = int(passes_away) if passes_away.isdigit() else None

                # Lấy thông tin đường chuyền chính xác
                if len(rows) > 8:
                    accurate_passes_home = rows[8].find_all('td')[1].text.strip()
                    accurate_passes_away = rows[8].find_all('td')[2].text.strip()
                    data["accurate_passes_home"] = int(accurate_passes_home) if accurate_passes_home.isdigit() else None
                    data["accurate_passes_away"] = int(accurate_passes_away) if accurate_passes_away.isdigit() else None

                # Lấy thông tin phạm lỗi
                if len(rows) > 10:
                    fouls_home = rows[10].find_all('td')[1].text.strip()
                    fouls_away = rows[10].find_all('td')[2].text.strip()
                    data["fouls_home"] = int(fouls_home) if fouls_home.isdigit() else None
                    data["fouls_away"] = int(fouls_away) if fouls_away.isdigit() else None

                # Lấy thông tin thẻ vàng
                if len(rows) > 14:
                    yellow_cards_home = rows[14].find_all('td')[1].text.strip()
                    yellow_cards_away = rows[14].find_all('td')[2].text.strip()
                    data["yellow_cards_home"] = int(yellow_cards_home) if yellow_cards_home.isdigit() else None
                    data["yellow_cards_away"] = int(yellow_cards_away) if yellow_cards_away.isdigit() else None

                # Lấy thông tin thời gian kiểm soát bóng
                if len(rows) > 13:
                    possession_time_home = rows[13].find_all('td')[1].text.strip()
                    possession_time_away = rows[13].find_all('td')[2].text.strip()
                    data["possession_time_home"] = possession_time_home
                    data["possession_time_away"] = possession_time_away

                # Lấy thông tin thẻ đỏ (nếu có)
                if len(rows) > 15:
                    red_cards_home = rows[15].find_all('td')[1].text.strip()
                    red_cards_away = rows[15].find_all('td')[2].text.strip()
                    data["red_cards_home"] = int(red_cards_home) if red_cards_home.isdigit() else None
                    data["red_cards_away"] = int(red_cards_away) if red_cards_away.isdigit() else None

            # Lấy tên đội và điểm số
            if score_span:
                score_text = score_span.text.strip()
                scores = score_text.split(" - ")
                data["home_score"] = int(scores[0]) if len(scores) > 0 and scores[0].isdigit() else None
                data["away_score"] = int(scores[1]) if len(scores) > 1 and scores[1].isdigit() else None
            else:
                data["home_score"] = None
                data["away_score"] = None
            
            if len(date_elements) >= 2:
                date_time = date_elements[1].text.strip()
                data["date"] = date_time
            else:
                data["date"] = None

    except Exception as e:
        print(f"Lỗi khi lấy thông tin từ link {link}: {e}")

    return data

# Đọc nội dung file lấy các link
with open("data_crawl/ss19_24.txt", "r") as file:
    links = file.read().splitlines()

# Sử dụng tqdm để theo dõi tiến trình cào dữ liệu
results = []

with ThreadPoolExecutor(max_workers=5) as executor:
    for data in tqdm(executor.map(scrape_link, links), total=len(links), desc="Đang cào dữ liệu"):
        results.append(data)

# Bây giờ bạn có thể xử lý các dữ liệu thu thập được trong `results`
print("Đã hoàn thành việc thu thập thông tin.")


Đang cào dữ liệu: 100%|██████████| 1894/1894 [06:38<00:00,  4.75it/s]

Đã hoàn thành việc thu thập thông tin.


In [4]:
import pandas as pd
# Chuyển đổi danh sách kết quả thành DataFrame
df = pd.DataFrame(results)

# Hiển thị DataFrame
df

# Lưu dữ liệu vào file CSV
df.to_csv("data_crawl/match_all_season_test.csv", index=False)

# lấy danh sách bảng điểm các mùa giải

In [11]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

# Danh sách các mùa giải
years = ['2019-2020', '2020-2021', '2021-2022', '2022-2023', '2023-2024']

# Khởi tạo danh sách để lưu toàn bộ dữ liệu
all_data = []

# Lặp qua các mùa giải với tqdm để hiển thị tiến độ
for year in tqdm(years, desc="Crawling data"):
    # URL tương ứng với từng mùa giải
    url = f"https://www.soccerstats247.com/competitions/england/premier-league/{year}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Tìm bảng có class và id cụ thể
    table = soup.find('table', {
        'class': 'gridAlternate hlCenter leaguesAll',
        'id': 'soccerContentPlaceHolder_ucChampionship_ucIndividualGroupStage_ucLeaguesTabPanel_gvLeagueTable'
    })

    # Kiểm tra nếu bảng tồn tại
    if table:
        # Lấy tất cả các hàng (tr), bỏ qua header (hàng đầu tiên)
        rows = table.find_all('tr')[1:]  # Bỏ header

        # Duyệt qua từng hàng và lấy dữ liệu từ các cột
        for row in rows:
            cols = row.find_all('td')
            cols = [col.get_text(strip=True) for col in cols]  # Lấy text từ từng cột
            cols.append(year)  # Thêm cột year
            all_data.append(cols)  # Thêm dữ liệu vào danh sách
    else:
        print(f"Không tìm thấy bảng cho mùa giải {year}.")

# Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(all_data, columns=["ID", "Team", "Played", "Won", "Draw", "Lost", "GF", "GA", "Goal Difference", "Points", "Last 5", "PPG", "% Points", "Year"])

# Hiển thị DataFrame
df


Crawling data: 100%|██████████| 5/5 [00:23<00:00,  4.66s/it]


,ID,Team,Played,Won,Draw,Lost,GF,GA,Goal Difference,Points,Last 5,PPG,% Points,Year
0,1,Liverpool,38,32,3,3,85,33,+52,99,,2.61,86.84,2019-2020
1,2,Manchester City,38,26,3,9,102,35,+67,81,,2.13,71.05,2019-2020
2,3,Manchester United,38,18,12,8,66,36,+30,66,,1.74,57.89,2019-2020
3,4,Chelsea,38,20,6,12,69,54,+15,66,,1.74,57.89,2019-2020
4,5,Leicester City,38,18,8,12,67,41,+26,62,,1.63,54.39,2019-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,16,Brentford,38,10,9,19,56,65,-9,39,,1.03,34.21,2023-2024
96,17,Nottingham Forest,38,9,9,20,49,67,-18,32,,0.95,31.58,2023-2024
97,18,Luton Town,38,6,8,24,52,85,-33,26,,0.68,22.81,2023-2024
98,19,Burnley,38,5,9,24,41,78,-37,24,,0.63,21.05,2023-2024


In [12]:
# xóa cột 'Last 5'
df = df.drop(columns=['Last 5'])

In [13]:
# check null 
df.isnull().sum()

ID                 0
Team               0
Played             0
Won                0
Draw               0
Lost               0
GF                 0
GA                 0
Goal Difference    0
Points             0
PPG                0
% Points           0
Year               0
dtype: int64

In [14]:
#lưu dữ liệu vào file csv
df.to_csv("data_crawl/points_all_season_test.csv", index=False)